In [ ]:
import pandas as pd

In [ ]:
folder = r"C:\GSP\sv\model\pest_inputs\PEST_PP"

In [ ]:
import os

filesls = []
for root, dirs, files in os.walk(folder):
    for file in files:
        if ".phi.group.csv" in file:
            filesls.append(os.path.join(root, file))

In [ ]:
dfall = []
cnt = 0
for f in filesls:

    df = pd.read_csv(os.path.join(f),index_col=[0,1,2,3])
    df = df.sum(axis =1).to_frame('phi')
    df.loc[:,'run'] = os.path.basename(f)+str(cnt)
    df = df.set_index('run', append = True)
    dfall.append(df)
    cnt+=1
    
dfall = pd.concat(dfall)
dfall = dfall.groupby(level = [4,0]).min().unstack()
dfall = dfall.div(dfall.min(axis = 1),axis = 'rows')

dfall = dfall.loc[dfall.count(axis = 1)>1]

In [ ]:
dfall.droplevel(0,1).query("run.str.contains('san') or run.str.contains('leo')").rename({"leo.phi.group.csv1":'experiment #2',
                                                                                             "sancho.phi.group.csv19":'experiment #1'}
                                                                                            ).dropna(how = 'all',axis = 1)

In [ ]:
ax = dfall.droplevel(0,1).query("run.str.contains('san') or run.str.contains('leo')").rename({"leo.phi.group.csv1":'experiment #2',
                                                                                             "sancho.phi.group.csv19":'experiment #1'}
                                                                                            ).dropna(how = 'all',axis = 1).plot.barh()
ax.grid(True)
ax.set_title('min phi reduction summary')
ax.set_xlabel('min phi, normalized')

In [ ]:
from dash import Dash, html, dcc, Input, Output, State
import plotly.graph_objs as go
import numpy as np

# Initialize the app
app = Dash(__name__)

# Sample data
x = np.linspace(0, 10, 50)
y = np.sin(x)

# Layout of the app
app.layout = html.Div([
    # Button to toggle menu visibility
    html.Button("Toggle Menu", id="toggle-button", n_clicks=0),
    # Hidden menu (initially hidden)
    html.Div(
        id="menu-bar",
        children=[
            html.Label("X-axis Range:"),
            dcc.Input(id="x-min", type="number", placeholder="Min", value=0),
            dcc.Input(id="x-max", type="number", placeholder="Max", value=10),
            html.Label("Y-axis Range:"),
            dcc.Input(id="y-min", type="number", placeholder="Min", value=-1),
            dcc.Input(id="y-max", type="number", placeholder="Max", value=1),
            html.Button("Apply", id="apply-button", n_clicks=0)
        ],
        style={"display": "none", "border": "1px solid black", "padding": "10px", "margin-bottom": "10px"}
    ),
    # Graph
    dcc.Graph(id="graph")
])

# Callback to toggle menu visibility
@app.callback(
    Output("menu-bar", "style"),
    [Input("toggle-button", "n_clicks")],
    [State("menu-bar", "style")]
)
def toggle_menu(n_clicks, style):
    if n_clicks % 2 == 1:
        return {"display": "block", "border": "1px solid black", "padding": "10px", "margin-bottom": "10px"}
    return {"display": "none"}

# Callback to update the graph based on inputs
@app.callback(
    Output("graph", "figure"),
    [Input("apply-button", "n_clicks")],
    [State("x-min", "value"),
     State("x-max", "value"),
     State("y-min", "value"),
     State("y-max", "value")]
)
def update_graph(n_clicks, x_min, x_max, y_min, y_max):
    # Create the figure with updated ranges
    fig = go.Figure(
        go.Scatter(x=x, y=y, mode='lines+markers', name="Sine Wave")
    )
    fig.update_layout(
        xaxis=dict(range=[x_min, x_max]),
        yaxis=dict(range=[y_min, y_max]),
        title="Interactive Plot with Collapsible Menu"
    )
    return fig

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)
